In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine
from utils import db_connect

pd.set_option('display.max_columns', None)

engine = db_connect()

# bring in the data
main_df = pd.read_sql('SELECT * FROM combined_data', con=engine)

# function to mark whether a school was operational all five years of data
# creates our 'Currently operational' feature
def mark_always_operational(main_df):
    
    total_years = main_df['SURVYEAR'].nunique()
    
    main_df['SY_STATUS_TEXT'] = main_df['SY_STATUS_TEXT'].str.strip()
    
    operational_counts = (
        main_df[main_df['SY_STATUS_TEXT'] == 'Currently operational']
        .groupby('NCESSCH')['SURVYEAR']
        .nunique()
    )
    
    always_operational_schools = operational_counts[operational_counts == total_years].index
    
    main_df['concurrently_operational'] = main_df['NCESSCH'].isin(always_operational_schools)
    
    return main_df

# apply the function
mark_always_operational(main_df=main_df)

# drop records that were not fully operational across all five years
main_df = main_df[main_df['concurrently_operational'] != False]

# strip whitespace
for col in main_df.select_dtypes(include=["string"]).columns:
    main_df[col] = main_df[col].str.strip()

# begin defining data type conversion processes
# Change columns to floats
float_cols = ["X", "Y", "LATCOD", "LONCOD", "FTE", "STUTERATIO"]

# change columns to int
int_cols = [
    "OBJECTID", "GSLO", "GSHI",
    "TOTFRL", "FRELCH", "REDLCH", "DIRECTCERT",
    "PK", "KG", "G01", "G02", "G03", "G04", "G05", "G06",
    "G07", "G08", "G09", "G10", "G11", "G12", "G13",
    "UG", "AE",
    "TOTMENROL", "TOTFENROL", "TOTAL", "MEMBER",
    "AMALM", "AMALF", "AM",
    "ASALM", "ASALF", "AS",
    "BLALM", "BLALF", "BL",
    "HPALM", "HPALF", "HP",
    "HIALM", "HIALF", "HI",
    "TRALM", "TRALF", "TR",
    "WHALM", "WHALF", "WH"
]

# Change columns to strings
string_cols = [
    "NCESSCH", "SURVYEAR", "STABR", "LEAID", "ST_LEAID",
    "LEA_NAME", "SCH_NAME",
    "LSTREET1", "LSTREET2", "LCITY", "LSTATE",
    "LZIP", "LZIP4", "PHONE",
    "VIRTUAL", "SCHOOL_LEVEL", "SCHOOL_TYPE_TEXT",
    "STATUS", "SY_STATUS_TEXT", "ULOCALE", "NMCNTY",
    "CHARTER_TEXT", "LSTREET3", "TITLEI", "STITLEI", "MAGNET_TEXT"
]
 
# -1 or M -> Indicates that the data are missing.

# -2 or N -> Indicates that the data are not applicable.

# -9 -> Indicates that the data do not meet NCES data quality standards.

# function to clean NCES error codes
def clean_nces_error_codes(main_df, cols):
    error_values = ["M", "-1", "-9", "Missing", -1, -9
]
    main_df[cols] = main_df[cols].replace(error_values, np.nan)
    return main_df

# clean ALL columns 
cols = float_cols + int_cols + string_cols
main_df = clean_nces_error_codes(main_df, cols)

# convert floats safely
for col in float_cols:
    main_df[col] = pd.to_numeric(main_df[col], errors="coerce")

    # convert ints safely
for col in int_cols:
    main_df[col] = pd.to_numeric(main_df[col], errors="coerce").astype("Int64")

    # convert strings
for col in string_cols:
    main_df[col] = main_df[col].astype("string")

    # round coordinates
main_df["LATCOD"] = main_df["LATCOD"].round(4)
main_df["LONCOD"] = main_df["LONCOD"].round(4)

# extract start Year - convert to int for sorting
main_df['SURVYEAR'] = main_df['SURVYEAR'].str[:4].astype(int)

# removing virtual schools
main_df = main_df[main_df['VIRTUAL'].isin(['Not Virtual', 'Not a virtual school'])]

# drop the virtual feature
main_df.drop(columns='VIRTUAL', inplace=True)

# only keeping 'regular' public schools, removing: [ 'Career and Technical School',
# 'Special education school', 'Alternative Education School',
# 'Alternative/other school', 'Vocational school']
main_df = main_df[main_df['SCHOOL_TYPE_TEXT'].isin(['Regular school', 'Regular School'])]

# drop the SCHOOL_TYPE_TEXT feature
main_df.drop(columns='SCHOOL_TYPE_TEXT', inplace=True)

# replace na values with 0
main_df = main_df.fillna(0)

# Checking records against all five years
counts = main_df["NCESSCH"].value_counts()
keep_ids = counts[counts == 5].index
main_df = main_df[main_df["NCESSCH"].isin(keep_ids)].copy()

print(f"1. main_df shape: {main_df.shape}")

# Simplify ULOCALE
main_df["locale_category"] = main_df["ULOCALE"].str.split("-").str[1].str.split(":").str[0]

# Drop the ULOCALE feature because we now have our simplified locale_category feature
main_df.drop(columns='ULOCALE', inplace=True)

# Title I rough breakdown:

# Participating:
# 1 - Yes - School participates in Title I funding / programs
# 5 - Title I schoolwide school - ENTIRE school recieves Title I support. Funds can be used for all students
# 2 - Title I targeted assistance school - Only SPECIFIC eligible students recieve services (usually low-income or academically at risk)

# Eligible, but no program running:
# 4 - Title I schoolwide eligible school - no program - Enough low-income students to qualify for schoolwide funding, but not using it
# 1 - Title I targeted assistance eligible school - No program - Eligible for targeted assistance but not participating

# Hybrid
# 3 Title I schoolwide eligible - Title I targeted assitance program - School qualifies for schoolwide funding but has chosen to run only a targeted program

# Explicit non-participation
# 2 - No - School does not participate
# 6 - Not a Title I school

# 0
# 0 - Assuming missing, unknown, or not reported


# Conceptual differences:
# Schoolwide = whole school qualifies = High funding flexibility - Typical poverty threshold >= 40% low-income
# Targeted = only some students qualify = Limited funding flexibility - lower threshold for poverty

# standardize TITLEI
schoolwide = ['1-Yes', '5-Title I schoolwide school']
targeted = ['2-Title I targeted assistance school', '3-Title I schoolwide eligible-Title I targeted assistance program']
elig_no_participate = ['4-Title I schoolwide eligible school-No program', 
                       '1-Title I targeted assistance eligible school-No program']
not_elig = ['2-No', '6-Not a Title I school']
missing = [0]

def group_titlei(col_TITLEI):
    if col_TITLEI in missing:
        return "Unknown"
    elif col_TITLEI in schoolwide:
        return "Schoolwide"
    elif col_TITLEI in targeted:
        return "Targeted"
    elif col_TITLEI in elig_no_participate:
        return "Eligible_No_Program"
    elif col_TITLEI in not_elig:
        return "Not_Eligible"
    else:
        return "Error"
    
# apply the above function to main_df
main_df['TITLEI_GROUPED'] = main_df['TITLEI'].apply(group_titlei)

# standardize STITLEI
STITLEI_yes = ['1-Yes', 'Yes']
STITLEI_no = ['2-No', 'No']
STITLEI_unknown = [0]

def standardize_STITLEI(col_STITLEI):
    if col_STITLEI in STITLEI_yes:
        return 'Yes'
    elif col_STITLEI in STITLEI_no:
        return 'No'
    elif col_STITLEI in STITLEI_unknown:
        return 'Unknown'
    else:
        return 'Error'
    
# apply the above function to main_df    
main_df['STITLEI'] = main_df['STITLEI'].apply(standardize_STITLEI)

# update the contradticions between TITLEI and STITLEI (Updating the below to 'Targeted' group instead of 'Schoolwide')
main_df.loc[(main_df['TITLEI'] == '1-Yes') & (main_df['STITLEI'] == 'No'), 'TITLEI_GROUPED'] = 'Targeted'

# Checking records against all five years
counts = main_df["NCESSCH"].value_counts()
keep_ids = counts[counts == 5].index
main_df = main_df[main_df["NCESSCH"].isin(keep_ids)].copy()

print(f"2. main_df shape: {main_df.shape}")

# further filtering on positive student teacher ratios
main_df = main_df[main_df['STUTERATIO'] != 0.0]

# define additional redundant columns
redundant_cols = ['X', 'Y', 'OBJECTID', 'ST_LEAID', 'LSTREET1', 'LSTREET2', 'LSTREET3', 
                  'LZIP4', 'PHONE', 'AMALM', 'AMALF', 'ASALM', 'ASALF', 
                  'BLALM', 'BLALF', 'HPALM', 'HPALF', 'HIALM', 'HIALF', 'TRALM', 'TRALF', 
                  'WHALM', 'WHALF', 'STABR', 'LCITY', 'LSTATE', 'LZIP', 'SCHOOL_LEVEL', 'GSLO', 'GSHI'
                  , 'STATUS', 'SY_STATUS_TEXT', 'NMCNTY', 'DIRECTCERT', 'AE', 'TOTFENROL', 'TOTMENROL',
                  'concurrently_operational', 'TITLEI', 'STITLEI', 'MEMBER']

# drop additional redundant cols
main_df = main_df.drop(columns=redundant_cols)

# remove the large Alaska homeschool support program from data set
main_df[main_df['NCESSCH'] != '20013000253']

# trim the top percentile off
def trim_top_percentile(df, col="STUTERATIO", percentile=0.99):

    df = df.copy()

    # Calculate cutoff
    cutoff = df[col].quantile(percentile)

    # Count rows before trimming
    before_count = df.shape[0]

    # Trim
    df_trimmed = df[df[col] <= cutoff].copy()

    after_count = df_trimmed.shape[0]

    print(f"{percentile*100}th percentile cutoff: {cutoff:.2f}")
    print(f"Rows before: {before_count}")
    print(f"Rows after: {after_count}")
    print(f"Rows removed: {before_count - after_count}")

    print("\nTop values after trimming:")
    print(
        df_trimmed.sort_values(col, ascending=False)[
            ["NCESSCH", "SURVYEAR", "FTE", col]
        ].head(10)
    )

    return df_trimmed

# apply the above function to main_df
main_df = trim_top_percentile(main_df, col="STUTERATIO", percentile=0.99)

# trim the bottom percentile off
def trim_bottom_percentile(df, col="STUTERATIO", percentile=0.01):

    df = df.copy()

    # Calculate cutoff
    cutoff = df[col].quantile(percentile)

    # Count rows before trimming
    before_count = df.shape[0]

    # Trim bottom values
    df_trimmed = df[df[col] >= cutoff].copy()

    after_count = df_trimmed.shape[0]

    print(f"{percentile*100}th percentile cutoff: {cutoff:.2f}")
    print(f"Rows before: {before_count}")
    print(f"Rows after: {after_count}")
    print(f"Rows removed: {before_count - after_count}")

    print("\nBottom values after trimming:")
    print(
        df_trimmed.sort_values(col, ascending=True)[
            ["NCESSCH", "SURVYEAR", "FTE", col]
        ].head(10)
    )

    return df_trimmed

main_df = trim_bottom_percentile(main_df)

# create our high-strain feature
main_df["high_strain"] = (main_df["STUTERATIO"] > 20).astype(int)

# updating nces error codes to No or 0 for respective columns
values = ["N", "-2","Not applicable", "Not Applicable",-2,]
main_df.loc[main_df['CHARTER_TEXT'].isin(values), 'CHARTER_TEXT'] = 'No'
main_df.loc[main_df['MAGNET_TEXT'].isin(values), 'MAGNET_TEXT'] = 'No'
main_df.loc[main_df['FRELCH'].isin(values), 'FRELCH'] = 0
main_df.loc[main_df['REDLCH'].isin(values), 'REDLCH'] = 0

# keeping all records with 5 years of data
counts = main_df["NCESSCH"].value_counts()
keep_ids = counts[counts == 5].index
main_df = main_df[main_df["NCESSCH"].isin(keep_ids)].copy()

print(f"3. main_df shape: {main_df.shape}")

In [ ]:
main_df.head(1)

In [ ]:
main_df.shape

In [ ]:
# main_df = pd.read_sql('SELECT * FROM main_df', con=engine)
main_df.to_sql("main_df", engine, if_exists="replace", index=False)

In [ ]:
main_df.head(1)

In [ ]:
sns.histplot(data=main_df, x='TOTAL')

In [ ]:
main_df['STUTERATIO'].describe()

In [ ]:
sns.histplot(data=main_df, x='STUTERATIO')

In [ ]:
# modeling EDA

# Do rural schools have higher strain?
# Do Title I schools have higher strain?
# Does enrollment size predict strain?
# Do funding indicators correlate?

In [ ]:
print(f"main_df shape: {main_df.shape}")
print(f"high strain schools: {main_df[main_df['high_strain'] == 1].shape}")

In [ ]:
main_df.head(1)

In [ ]:
keep_cols = ['SURVYEAR', 'LEAID', 'CHARTER_TEXT', 'MAGNET_TEXT', 'TOTFRL', 'FRELCH', 'REDLCH', 
             'PK', 'KG', 'G01', 'G02', 'G03', 'G04', 'G05', 'G06', 'G07', 'G08',
               'G09', 'G10', 'G11', 'G12', 'G13', 'UG', 'TOTAL', 'AM', 'AS', 'BL', 'HP', 'HI', 'TR'
               , 'WH', 'LATCOD', 'LONCOD', 'locale_category', 'TITLEI_GROUPED', 'high_strain']
refined_df = main_df[keep_cols]

In [ ]:
# for corr
subset_cols = ['high_strain', 'LEAID', 'TOTAL', 'locale_category', 'TITLEI_GROUPED']
subset_df = main_df[subset_cols]

In [ ]:
subset_cols = ['TOTAL', 'AM', 'AS', 'BL', 'HP', 'HI', 'TR'
               , 'WH', 'LATCOD', 'LONCOD','high_strain', 'TOTFRL']
subset_df = main_df[subset_cols]

In [ ]:
subset_df.head(1)

In [ ]:
cat_cols = subset_df.select_dtypes(include=['category', 'object']).columns

# Keep numeric columns
num_cols = subset_df.select_dtypes(include=['number']).columns

df_encoded_full = pd.get_dummies(subset_df, columns=cat_cols)

print(df_encoded_full.head())

In [ ]:
correlation_matrix = df_encoded_full.corr()

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, 
            annot=True,        # show the correlation numbers
            fmt=".2f",         # 2 decimal places
            cmap="coolwarm",   # color scheme
            cbar=True)
plt.title("Correlation Matrix of Categorical Features")
plt.show()

In [ ]:
df_encoded_full.shape

In [ ]:
main_df.columns

# Logistic Regression preprocessing

In [ ]:
# Ethnicity proportions
for col in ['AM','AS','BL','HP','HI','TR','WH']:
    main_df[f'prop_{col}'] = main_df[col] / main_df['TOTAL']

# Grade-level ratios
main_df['upper_grades_ratio'] = main_df[['G09','G10','G11','G12','G13']].sum(axis=1) / main_df['TOTAL']
main_df['lower_grades_ratio'] = main_df[['PK','KG','G01','G02','G03','G04','G05']].sum(axis=1) / main_df['TOTAL']
main_df['middle_grades_ratio'] = main_df[['G06','G07','G08']].sum(axis=1) / main_df['TOTAL']

# Free/reduced lunch ratios
main_df['frl_ratio'] = main_df['FRELCH'] / main_df['TOTFRL']
main_df['redl_ratio'] = main_df['REDLCH'] / main_df['TOTFRL']


In [ ]:
main_df.head(1)

In [ ]:
cols_to_drop = ['AM','AS','BL','HP','HI','TR','WH', 'G06','G07','G08','G09','G10','G11','G12','G13', 'PK','KG','G01','G02','G03','G04','G05', 'TOTFRL']

In [ ]:
main_df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
main_df.head(1)

In [ ]:
main_df['CHARTER_TEXT'].unique()

In [ ]:
# main_df.loc[main_df['REDLCH'].isin(values), 'REDLCH'] = 0
main_df.loc[main_df['CHARTER_TEXT'] == 0, 'CHARTER_TEXT'] = 'Missing'
main_df.loc[main_df['MAGNET_TEXT'] == 0, 'MAGNET_TEXT'] = 'Missing'

In [ ]:
main_df.head(1)

In [ ]:
main_df.columns

In [ ]:
main_df.drop(columns=['NCESSCH', 'SURVYEAR', 'LEA_NAME', 
                      'SCH_NAME', 'FRELCH', 'REDLCH', 'FTE', 'STUTERATIO'], inplace=True)

In [ ]:
cat_cols=main_df.select_dtypes(include=['object','category']).columns
cat_cols

In [ ]:
main_df = pd.get_dummies(
    main_df,
    columns=main_df.select_dtypes(include=['object','category']).columns,
    drop_first=True
)

In [ ]:
main_df.head(5)

In [ ]:
# When I did the free and reduced lunch convertions, it turned 0's into NAs so need to update to 0's
main_df.loc[main_df['frl_ratio'].isna(), 'frl_ratio'] = 0
main_df.loc[main_df['redl_ratio'].isna(), 'redl_ratio'] = 0

In [ ]:
X = main_df.drop(columns=["high_strain"])
y = main_df["high_strain"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
# Unsure if scaling is the correct path with this dataset... might revist this with scaling.

In [ ]:
main_df[main_df.isna().any(axis=1)]

In [ ]:
# baseline logreg model
from sklearn.linear_model import LogisticRegression

base_model = LogisticRegression(max_iter=500)
base_model.fit(X_train, y_train)

In [ ]:
base_model_y_pred = base_model.predict(X_test)
base_model_y_prob = base_model.predict_proba(X_test)[:,1]

In [ ]:
# evaluating base logreg model
# as thought, this model is essentially guessing, with a horrible recall of actually determining strain
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, base_model_y_pred))
print("ROC AUC:", roc_auc_score(y_test, base_model_y_prob))
print(classification_report(y_test, base_model_y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l2"],
    "solver": ["lbfgs", "liblinear"]
}

grid = GridSearchCV(
    LogisticRegression(class_weight='balanced', max_iter=500),
    param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1
)

grid.fit(X_train, y_train)

In [ ]:
cv_model = grid.best_estimator_
print(grid.best_params_)

In [ ]:
cv_y_pred = cv_model.predict(X_test)
cv_y_prob = cv_model.predict_proba(X_test)[:,1]

In [ ]:
# evaluating best model from cv 
print("Accuracy:", accuracy_score(y_test, cv_y_pred))
print("ROC AUC:", roc_auc_score(y_test, cv_y_prob))
print(classification_report(y_test, cv_y_pred))

In [ ]:
coef_df = pd.DataFrame({
    "feature": X.columns,
    "coef": cv_model.coef_[0]
}).sort_values("coef", ascending=False)

In [ ]:
coef_df

In [ ]:
# interpretation to come of the above ceofficients, I still want to try 
# to scale some things out as well and see if our coefficients change at all.
# moving to looking at additional data to bring in.

Looking at additional data set for income information broken out by LEAID (school district)

In [ ]:
addi_df = pd.read_csv('../data/raw/DP03_001_USSchoolDistrictAll_219154124322.txt', sep="|", dtype=str)

print(addi_df.head())
print(addi_df.shape)

In [ ]:
addi_df.head(1)

In [ ]:
# econ_df = pd.read_sql('SELECT * FROM econ_2018_2022_rolling', con=engine)
addi_df.to_sql("econ_2018_2022_rolling", engine, if_exists="replace", index=False)

In [ ]:
addi_df.shape

In [ ]:
import requests

url = "https://api.census.gov/data/2022/acs/acs5/profile/groups/DP03.json"

response = requests.get(url)

print(response.status_code)  # Should be 200
print(response.headers.get('Content-Type'))  # Should indicate JSON

if response.status_code == 200:
    try:
        data = response.json()
        print(data.keys())  # Should show keys like 'variables', 'name', 'description'
    except ValueError as e:
        print("JSON decode error:", e)
        print("Response text preview:", response.text[:500])
else:
    print("Request failed with status code:", response.status_code)

In [ ]:
data

In [ ]:
variables = data['variables']

# Convert to DataFrame
var_df = pd.DataFrame.from_dict(variables, orient='index').reset_index()
var_df = var_df.rename(columns={'index': 'variable'})

In [ ]:
var_df.head(1)

In [ ]:
var_df.shape

In [ ]:
# econ_var_def_df = pd.read_sql('SELECT * FROM econ_var_def_2018_2022_rolling', con=engine)
var_df.to_sql("econ_var_def_2018_2022_rolling", engine, if_exists="replace", index=False)

In [ ]:
addi_df.head(1)

In [ ]:
addi_cols = addi_df.columns

In [ ]:
type(addi_cols)

In [ ]:
addi_cols = pd.Series(addi_cols)

In [ ]:
type(addi_cols)

In [ ]:
addi_cols = list(addi_cols)

In [ ]:
type(addi_cols)

In [ ]:
var_df[var_df['variable'].isin(addi_cols)]

In [ ]:
# After some research the initial data set contained the non-official, human firendly col names
# however the API call to gather the definitions are the official labels, meaning we have a mismatch,
# trying to determine a solid option for convertion

In [ ]:
import re

# suppose addi_cols = ['DP03_133pct', 'DP03_133pctmoe', ...]
mapped_cols = []
for col in addi_cols:
    # extract the number part
    m = re.search(r'DP03_(\d+)', col)
    if m:
        rownum = m.group(1).zfill(4)  # API variables are zero-padded 4 digits
        if col.endswith('pct'):
            mapped_cols.append(f'DP03_{rownum}PE')
        elif col.endswith('pctmoe'):
            mapped_cols.append(f'DP03_{rownum}M')
            
print(mapped_cols)

In [ ]:
var_df[var_df['variable'].isin(mapped_cols)]

In [ ]:
# Success!

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
var_df[var_df['variable'].isin(mapped_cols)]

In [ ]:
# addi_df = the new economic data set (2018-2022 rolling 5 year)
# var_df = API call to the data dictionary for the DP03 economic variable list.

In [ ]:
addi_df.head(1)

In [ ]:
var_df.head(1)

In [ ]:
addi_df['DP03_0053EA'].head(1)

In [ ]:
# I actually want the inverse idea from above, to be able to bring addi_df into the db
def api_to_addi(api_cols):
    converted = []
    
    for col in api_cols:
        m = re.search(r'DP03_(\d+)(PE|M)$', col)
        if m:
            num = str(int(m.group(1)))  # remove zero padding
            suffix = m.group(2)
            
            if suffix == "PE":
                converted.append(f"DP03_{num}pct")
            elif suffix == "M":
                converted.append(f"DP03_{num}pctmoe")
    
    return converted

In [ ]:
var_df['label'].head(1)

In [ ]:
income_vars = var_df[
    var_df['label'].str.contains('income', case=False, na=False)
]['variable']

In [ ]:
income_vars = list(income_vars)

In [ ]:
income_vars = api_to_addi(income_vars)

In [ ]:
addi_df.head(1)

In [ ]:
cols_to_remove = [
    'DP03_71pct', 'DP03_69pctmoe', 'DP03_67pctmoe', 'DP03_63pctmoe',
    'DP03_65pctmoe', 'DP03_62pctmoe', 'DP03_67pct', 'DP03_90pct',
    'DP03_73pctmoe', 'DP03_75pctmoe', 'DP03_71pctmoe', 'DP03_51pct',
    'DP03_88pctmoe', 'DP03_89pctmoe', 'DP03_86pctmoe', 'DP03_87pctmoe',
    'DP03_93pctmoe', 'DP03_94pctmoe', 'DP03_91pctmoe', 'DP03_92pctmoe',
    'DP03_90pctmoe', 'DP03_65pct', 'DP03_88pct', 'DP03_62pct',
    'DP03_75pct', 'DP03_93pct', 'DP03_89pct', 'DP03_63pct',
    'DP03_94pct', 'DP03_86pct', 'DP03_73pct', 'DP03_91pct',
    'DP03_69pct', 'DP03_87pct', 'DP03_92pct', 'DP03_51pctmoe'
]

income_vars = [c for c in income_vars if c not in cols_to_remove]

In [ ]:
addi_df[income_vars].head(1)

In [ ]:
var_df[var_df['variable'].isin(income_vars)]

In [ ]:
var_df.shape

In [ ]:
# at this point I need to just build a crossover mapping table between the two
import re
rows = []

for var in var_df['variable']:
    m = re.match(r'DP03_(\d+)(PE|M)$', var)
    if m:
        num = str(int(m.group(1)))
        suffix = m.group(2)

        if suffix == "PE":
            addi = f"DP03_{num}pct"
        else:
            addi = f"DP03_{num}pctmoe"

        rows.append({
            "api_var": var,
            "addi_col": addi,
            "label": var_df.loc[var_df.variable == var, "label"].values[0],
            "concept": var_df.loc[var_df.variable == var, "concept"].values[0]
        })

map_df = pd.DataFrame(rows)

In [ ]:
map_df.head(1)

In [ ]:
map_df.to_sql("acs_column_map", engine, if_exists="replace", index=False)

In [ ]:
addi_df.head(1)

In [ ]:
cols_to_check = ['DP03_48pct','DP03_22pct','DP03_111pct','DP03_35pct','DP03_66pct','DP03_79pct','DP03_53pct','DP03_97pct','DP03_40pct','DP03_71pct','DP03_84pct','DP03_107pct','DP03_5pct','DP03_125pct','DP03_18pct','DP03_49pct','DP03_36pct','DP03_112pct','DP03_23pct','DP03_67pct','DP03_10pct','DP03_41pct','DP03_130pct','DP03_54pct','DP03_85pct','DP03_98pct','DP03_108pct','DP03_72pct','DP03_19pct','DP03_113pct','DP03_6pct','DP03_90pct','DP03_126pct','DP03_59pct','DP03_2pct','DP03_135pct','DP03_46pct','DP03_122pct','DP03_20pct','DP03_33pct','DP03_64pct','DP03_77pct','DP03_118pct','DP03_51pct','DP03_105pct','DP03_52pct','DP03_95pct','DP03_136pct','DP03_29pct','DP03_82pct','DP03_3pct','DP03_16pct','DP03_110pct','DP03_47pct','DP03_123pct','DP03_34pct','DP03_78pct','DP03_21pct','DP03_65pct','DP03_106pct','DP03_83pct','DP03_119pct','DP03_96pct','DP03_137pct','DP03_17pct','DP03_70pct','DP03_124pct','DP03_4pct','DP03_133pct','DP03_13pct','DP03_120pct','DP03_57pct','DP03_44pct','DP03_88pct','DP03_31pct','DP03_62pct','DP03_9pct','DP03_129pct','DP03_75pct','DP03_93pct','DP03_116pct','DP03_27pct','DP03_80pct','DP03_103pct','DP03_1pct','DP03_121pct','DP03_14pct','DP03_45pct','DP03_134pct','DP03_58pct','DP03_89pct','DP03_32pct','DP03_76pct','DP03_117pct','DP03_63pct','DP03_50pct','DP03_28pct','DP03_81pct','DP03_94pct','DP03_104pct','DP03_15pct','DP03_24pct','DP03_37pct','DP03_68pct','DP03_100pct','DP03_11pct','DP03_55pct','DP03_109pct','DP03_131pct','DP03_99pct','DP03_42pct','DP03_86pct','DP03_7pct','DP03_60pct','DP03_73pct','DP03_114pct','DP03_101pct','DP03_127pct','DP03_38pct','DP03_91pct','DP03_132pct','DP03_25pct','DP03_69pct','DP03_12pct','DP03_43pct','DP03_56pct','DP03_87pct','DP03_30pct','DP03_74pct','DP03_128pct','DP03_61pct','DP03_8pct','DP03_92pct','DP03_102pct','DP03_26pct','DP03_115pct','DP03_39pct']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine
from utils import db_connect
engine = db_connect()
main_econ_df = pd.read_sql('SELECT * FROM main_econ_view', con=engine)

In [ ]:
main_econ_df['LEAID'] = main_econ_df['LEAID'].astype(int)

In [ ]:
main_econ_df['SURVYEAR'].unique()

In [ ]:
main_econ_df.head()

In [ ]:
# PREPROCESSING
# Ethnicity proportions
for col in ['AM','AS','BL','HP','HI','TR','WH']:
    main_econ_df[f'prop_{col}'] = main_econ_df[col] / main_df['TOTAL']

# Grade-level ratios
main_econ_df['upper_grades_ratio'] = main_econ_df[['G09','G10','G11','G12','G13']].sum(axis=1) / main_econ_df['TOTAL']
main_econ_df['lower_grades_ratio'] = main_econ_df[['PK','KG','G01','G02','G03','G04','G05']].sum(axis=1) / main_econ_df['TOTAL']
main_econ_df['middle_grades_ratio'] = main_econ_df[['G06','G07','G08']].sum(axis=1) / main_econ_df['TOTAL']

# Free/reduced lunch ratios
main_econ_df['frl_ratio'] = main_econ_df['FRELCH'] / main_econ_df['TOTFRL']
main_econ_df['redl_ratio'] = main_econ_df['REDLCH'] / main_econ_df['TOTFRL']

cols_to_drop = ['AM','AS','BL','HP','HI','TR','WH', 'G06','G07','G08','G09','G10','G11','G12','G13', 'PK','KG','G01','G02','G03','G04','G05', 'TOTFRL']

main_econ_df.drop(columns=cols_to_drop, inplace=True)

main_econ_df.loc[main_econ_df['CHARTER_TEXT'] == 0, 'CHARTER_TEXT'] = 'Missing'
main_econ_df.loc[main_econ_df['MAGNET_TEXT'] == 0, 'MAGNET_TEXT'] = 'Missing'

In [ ]:
main_econ_df.head(1)

In [ ]:
main_econ_df.drop(columns=['NCESSCH', 'SURVYEAR', 'LEA_NAME', 
                      'SCH_NAME', 'FRELCH', 'REDLCH', 'FTE', 'STUTERATIO'], inplace=True)

In [ ]:
cat_cols=main_econ_df.select_dtypes(include=['object','category']).columns
cat_cols

In [ ]:
main_econ_df = pd.get_dummies(
    main_econ_df,
    columns=main_econ_df.select_dtypes(include=['object','category']).columns,
    drop_first=True
)

In [ ]:
main_econ_df.loc[main_econ_df['frl_ratio'].isna(), 'frl_ratio'] = 0
main_econ_df.loc[main_econ_df['redl_ratio'].isna(), 'redl_ratio'] = 0

In [ ]:
X = main_econ_df.drop(columns=["high_strain"])
y = main_econ_df["high_strain"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
main_econ_df[main_econ_df.isna().any(axis=1)]

In [ ]:
# organization
# visualizations
# figure out tableau (or streamlit)
# go through presentation of ideas (what does it all mean)
# double down on API files iteration
# 